# 2022/2/1（鈴木・第７回）二分探索、繰り返し二乗法

今日のテーマは、計算量が $O(\log{n})$ となるアルゴリズム

## テーマ１：二分探索

## 例題: 年齢あてゲーム

- デーモンさんの年齢を当てたい。
- 1 歳以上 100 万歳以下であることはわかっている
- 「X 歳以上ですか？」質問すると、Yes か No かで答えてくれる
- 何回質問すればよいか？

**解答１**

「1 歳以上ですか？」「2 歳以上ですか？」「3 歳以上ですか？」...

正解が X 歳の場合、X+1 回目の質問で初めて No が返ってくるので、最悪の質問回数は 100万回

**解答 ２**

「50 万歳以上ですか？」「25 万歳以上ですか？」「12.5 万歳以上ですか？」..

年齢としてありうる区間を半分ずつに狭めているので、質問の最大回数は $\log_2(10^6) \le 20$.

たった 20 回で年齢を当てることができる！














## 例題：方程式の解

$f(x) = x^2-2$ の正の実数解の近似値を一つ求めよ。

**アイデア：二分法、区間縮小法**

区間 $[L,R]$ について、$f(L) < 0$, $f(R) > 0$ とする。$M = (L+R)/2$ とする。このとき

- もし $f(M) < 0$ なら $f(M) < 0$, $f(R) > 0$ がみたされるので区間 $[M,R]$ に解がある
- もし $f(M) > 0$ なら $f(L) < 0$, $f(M) > 0$ がみたされるので区間 $[L,M]$ に解がある

よって解が存在する区間の幅を、１回の計算で半分にできる。
50 回ほど計算すれば、区間幅は $2^{-50} = 10^{-15}$ 倍になるので、十分な精度だろう。

今回は $f(1) < 0 < f(2)$ なので、初期区間を $[1,2]$ として計算すればよい。




In [1]:
def f(x):
    return x**2 - 2

L,R = 1,2
for _ in range(100):
    M = (L+R)/2
    if f(M) < 0:
        L = M
    else:
        R = M
# M が近似解
print(M)
print(f(M))


1.414213562373095
-4.440892098500626e-16


## 例題：ソートされた区間の二分探索

$A$ をソートされたリストとする。
数 $x$ が与えられるので、 $x$ が $A$ に存在するかどうかを判定せよ。

### 解答例

年齢あてゲームと同じ。
「$A[i]$ は $x$ 以上ですか？」という質問をして、区間を半分に区切っていく。
$A$ の長さを $n$ とすると、$log_2(n)$ 回の質問で答えが求まる。

ここで、python の `bisect` ライブラリを使うと、
ソートされた区間の二分探索ができる。

$x \le A[i]$ となる最小の $i$ は `i = bisect.bisect_left(A,x)` で求まる。（そのような $i$ が存在しないとき、$i$ の値は `len(a)` となる。この場合、$A[i]$は領域外参照となることに注意）


**例題**
`A = [2,3,5,7,11,14,17,19]` に $x$ はあるか？


In [2]:
from bisect import bisect_left

A = [2,3,5,7,11,14,17,19]
def f(x):
    idx = bisect_left(A,x)
    if idx < len(A) and A[idx] == x:
        return True
    else:
        return False

print(f(9))
print(f(11))
print(f(19))
print(f(20))

False
True
True
False


## テーマ２：繰り返し二乗法（バイナリ法）

**問題**

$2^{10^{18}}$ を $12345$ で割った余りを求めよ。

**解答１：python の組み込み関数を使う**

`pow(2,10**18)` の計算は、さすがに不可能。

実は、python の pow 関数は３つ引数をとる使い方ができる。
`pow(a,n,m)` とすると、まさに $a^n$ を $m$ で割った余りが高速に求まる。


In [3]:
pow(2,10**18,12345)

4396

なぜこんなに高速に計算できるのだろうか？

## 繰り返し二乗法

**アイデア:** $a,a^2,a^4,a^8,a^{16}, \cdots$ は二乗を繰り返すことで計算できる。

$f(n) = a^n$ とおく。以下の等式は mod $m$ で考える。

$$
f(n) = a^n = \begin{cases}
f(n/2)^2 & \text{if $n$: even}\\
a f(n-1) & \text{if $n$: odd}
\end{cases}
$$

という再帰で計算する。すると、「再帰を二回実行すると $n$ は半分以下になる」ので、$2 \log_2{n}$ 回の計算で値が求まる！


In [4]:
# f(a,n,m) = a^n mod m
def f(a,n,m):
    # ベースケース
    if n == 0:
        return 1%m
    # 偶数のとき
    if n%2 == 0:
        return (f(a,n//2,m)**2)%m
    # 奇数のとき
    else:
        return a*f(a,n-1,m)%m

print(f(2,10**18,12345))

4396


## 練習問題

方程式 $x^3 + x - 1 = 0$ の実数解を数値的に求めよ。


In [ ]:
# 解答欄










**解答例**

In [11]:
def f(x):
    return x**3 + x - 1

# グラフの形を考えると、実数解をひとつだけ持つことがわかる
# f(0) < 0 < f(1) なので解は区間 [0,1] の中にある

L,R = 0,1
for _ in range(100):
    M = (L+R)/2
    if f(M) < 0:
        L = M
    else:
        R = M
# M が近似解
print(M)
print(f(M))

0.6823278038280194
2.220446049250313e-16


## 練習問題

ソート済みのリスト $A$ が事前に与えられる。
$f(x)$ を「$A$ の中にある、$x$ 以下の元の個数」と定める。

- $f(x)$ を実装せよ。ただし計算量が $O(\log(A の長さ))$ となるようにせよ。
- `A = list(range(10**5))` として、以下を確かめよ
    - $f(-1) = 0$
    - $f(0) = 1$
    - $f(100) = 101$
    - $f(99999) = 10^5$
    - $f(100009) = 10^5$

**ヒント**
`bisect` を使う。

In [ ]:
# 解答欄








# 以下、関数値の確認用に使ってもよい

# A = list(range(10**5))
# for v in [-1,0,100,99999,100009]:
#    print(f(v))


**解答例**

- `bisect_left(A,x)` は $x$ 「未満」の元の個数を返す
- よって `bisect_left(A,x+1)` とする必要がある
- もしくは、（説明していないが）`bisect_right(A,x)` としてもよい

In [18]:
from bisect import bisect_left
def f(x):
    idx = bisect_left(A,x+1)
    return idx

A = list(range(10**5))
for v in [-1,0,100,99999,100009]:
    print(f(v))


0
1
101
100000
100000


## 練習問題

$x \circ y := xy + x + y$ と定める。
$f(x,n) = x \circ \cdots \circ x$ と定める。形式的には
$$
\begin{cases}
f(x,0) = 0 & \text{if $n = 0$}, \\
f(x,n) = f(x,n-1) \circ x & \text{if $n > 0$},
\end{cases}
$$
と定める。たとえば $f(2,3) = 2 \circ 2 \circ 2 = 8 \circ 2 = 26$ である。

- $f(x,n)$ を $m$ で割った余りを求める関数 $g(x,n,m)$ を実装せよ。
- $f(2,10^{18},12345) = 10910$  を確かめよ。

**ヒント?**
- じつは演算 $\circ$ は結合律をみたす。
- また $0$ は単位元である、よって演算 $\circ$ によりモノイドとなる
- 繰り返し二乗法のアイデアはモノイドに対して適用できる。


In [ ]:
# 解答欄








**解答例**

In [14]:
def f(x,y):
    return x*y + x + y

def g(x,n,m):
    if n == 0: # n=0 のときはモノイドの単位元を返す
        return 0
    if n%2 == 0:
        v = g(x,n//2,m)
        return f(v,v)%m
    else:
        v = g(x,n-1,m)
        return f(v,x)%m

print(g(2,2**18,12345))

# 実は、以下の関数も正しい解を返すことが数学的に証明できる
# なぜだろうか？

def h(x,n,m):
    return (pow(x+1,n,m) - 1) % m

for x,n,m in [(2,10**18,12345),(3,10**9,34567),(13435,134245,4837462736)]:
    assert g(x,n,m) == h(x,n,m)


10910
